In [0]:
# Importing the necessary libraries
import os
import pandas as pd
from skimage.util import img_as_float
from skimage import io
from sklearn.metrics import jaccard_score, f1_score, precision_score, recall_score, accuracy_score, confusion_matrix

In [0]:
# Function to binarize an array based on a given threshold
def binarize(array, threshold=0.5):
    return (array > threshold).astype(int)

# Function to calculate the values of the confusion matrix
def calculate_confusion_matrix_values(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    return tp, tn, fp, fn

In [7]:
# Paths to segmentation files and ground truth files
path_to_segmentations = "PATH TO PREDICTED SEGMENTATION FILES"
path_to_ground_truth = "PATH TO GROUND TRUTH SEGMENTATION FILES"

# List the segmentation and ground truth files
segmentation_files = os.listdir(path_to_segmentations)
ground_truth_files = os.listdir(path_to_ground_truth)

# Initialize a dictionary to store various metrics for each file
metrics = {
    'file': [], 'dice': [], 'jaccard': [], 'precision': [],
    'recall': [], 'accuracy': [], 'TP': [], 'TN': [], 'FP': [], 'FN': []
}

# Loop through the files and calculate the metrics
for index in range(len(segmentation_files)):
    # Construct full file paths
    segmentation_file_name = path_to_segmentations + segmentation_files[index]
    ground_truth_file_name = path_to_ground_truth + ground_truth_files[index]

    # Read and process (convert to float) the images
    segmentation_image = img_as_float(io.imread(segmentation_file_name, as_gray=True))
    ground_truth_mask = img_as_float(io.imread(ground_truth_file_name, as_gray=True))

    # Binarize the images
    segmentation_image = binarize(segmentation_image)
    ground_truth_mask = binarize(ground_truth_mask)

    # Flatten the images to 1D for metric calculations
    segmentation_flat = segmentation_image.flatten()
    ground_truth_flat = ground_truth_mask.flatten()

    # Compute different metrics
    dice = f1_score(ground_truth_flat, segmentation_flat)
    jaccard = jaccard_score(ground_truth_flat, segmentation_flat)
    precision = precision_score(ground_truth_flat, segmentation_flat)
    recall = recall_score(ground_truth_flat, segmentation_flat)
    accuracy = accuracy_score(ground_truth_flat, segmentation_flat)
    tp, tn, fp, fn = calculate_confusion_matrix_values(ground_truth_flat, segmentation_flat)

    # Append metrics to the dictionary
    metrics['file'].append(segmentation_files[index])
    metrics['dice'].append(dice)
    metrics['jaccard'].append(jaccard)
    metrics['precision'].append(precision)
    metrics['recall'].append(recall)
    metrics['accuracy'].append(accuracy)
    metrics['TP'].append(tp)
    metrics['TN'].append(tn)
    metrics['FP'].append(fp)
    metrics['FN'].append(fn)

# Create DataFrame from the metrics and calculate the average
metrics_dataframe = pd.DataFrame(metrics)
average_metrics = metrics_dataframe[['dice', 'jaccard', 'precision', 'recall', 'accuracy', 'TP', 'TN', 'FP', 'FN']].mean(numeric_only=True)
average_row = ['Average'] + average_metrics.tolist()

# Append average row to DataFrame
metrics_dataframe.loc[len(metrics_dataframe)] = average_row
metrics_dataframe = metrics_dataframe.round(4)

# Save the DataFrame to an Excel file
metrics_dataframe.to_excel("/metrics.xlsx", index=False)